# SkillAssessGPT - Générateur d'Évaluations APC

Ce notebook vous guide à travers le processus de génération automatique de grilles d'évaluation selon l'Approche Par Compétences (APC).

## Vue d'ensemble

SkillAssessGPT utilise l'intelligence artificielle pour créer :
- **Grilles APC structurées** avec les niveaux ND, NI, et NA
- **Situations d'évaluation authentiques** adaptées au contexte
- **Barèmes de notation** cohérents et équilibrés
- **Validation pédagogique** automatique par un agent critique

## Architecture

```
Entrées utilisateur → Agent Générateur (LLM) → Agent Critique (LLM) → Export (JSON/Markdown)
```

## 1. Configuration et Imports

Commençons par importer les modules nécessaires et configurer l'environnement.

In [1]:
# Imports
import os
import sys
from dotenv import load_dotenv
import json
from IPython.display import display, Markdown, HTML
import pandas as pd

# Add src to path
sys.path.insert(0, os.path.abspath('.'))

from src.models import CompetencyInput, AssessmentOutput, ValidationResult
from src.generator import GeneratorAgent
from src.critic import CriticAgent
from src.exporter import ExportModule

# Load environment variables
load_dotenv()

print("✅ Modules importés avec succès!")

✅ Modules importés avec succès!


C:\Users\ramib\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Configuration de l'API

Configurez votre clé API Google Gemini. Vous pouvez soit :
- La définir dans un fichier `.env` : `GOOGLE_API_KEY=votre_clé`
- La définir directement dans la cellule ci-dessous (non recommandé pour la production)

In [2]:
# Récupérer la clé API
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    print("⚠️  Clé API non trouvée dans les variables d'environnement.")
    print("Veuillez créer un fichier .env avec : GOOGLE_API_KEY=votre_clé")
else:
    print("✅ Clé API chargée avec succès!")
    print(f"   Clé : {api_key[:8]}...{api_key[-4:]}")

✅ Clé API chargée avec succès!
   Clé : AIzaSyDm...9FOU


## 3. Définir les Paramètres d'Entrée

Définissez la compétence à évaluer et les paramètres contextuels.

### Exemple : Développement Web

In [3]:
# Définir les paramètres d'entrée
competency_input = CompetencyInput(
    competency="Développer une application web responsive",
    element="Créer une interface utilisateur moderne et accessible",
    niveau="Licence 2",
    parcours="Informatique",
    specialite="Développement Web",
    duree="2 heures"
)

print("📝 Paramètres d'entrée définis :")
print(f"   Compétence : {competency_input.competency}")
print(f"   Niveau : {competency_input.niveau}")
print(f"   Parcours : {competency_input.parcours}")
print(f"   Spécialité : {competency_input.specialite}")
print(f"   Durée : {competency_input.duree}")

📝 Paramètres d'entrée définis :
   Compétence : Développer une application web responsive
   Niveau : Licence 2
   Parcours : Informatique
   Spécialité : Développement Web
   Durée : 2 heures


### Alternative : Saisie Interactive

Vous pouvez également utiliser l'interface de collecte interactive :

In [5]:
# Décommentez pour utiliser la saisie interactive
# from src.input_collector import InputCollector
# collector = InputCollector()
# competency_input = collector.collect_inputs()

## 4. Génération de la Grille APC

Utilisons l'agent générateur pour créer la grille d'évaluation, la situation authentique, et le barème.

⏱️ Cette étape prend généralement 10-20 secondes.

In [4]:
# Initialiser l'agent générateur
generator = GeneratorAgent(api_key)

print("🤖 Génération de la grille d'évaluation en cours...")
print("   Cela peut prendre 10-20 secondes...\n")

# Générer l'évaluation
assessment = generator.generate_assessment(competency_input)

print("\n✅ Grille d'évaluation générée avec succès!")
print(f"   - {len(assessment.grid.nd_criteria)} critères ND (Non Développé)")
print(f"   - {len(assessment.grid.ni_criteria)} critères NI (Niveau Intermédiaire)")
print(f"   - {len(assessment.grid.na_criteria)} critères NA (Niveau Attendu)")
print(f"   - Barème total : {assessment.rubric.total_points} points")

🤖 Génération de la grille d'évaluation en cours...
   Cela peut prendre 10-20 secondes...

API call failed (attempt 1/3): API request failed: 404 models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Retrying in 1 seconds...
API call failed (attempt 2/3): API request failed: 404 models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Retrying in 2 seconds...


Exception: Failed to generate assessment after 3 attempts: API request failed: 404 models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

## 5. Visualisation de la Grille Générée

Affichons la grille APC sous forme de tableaux interactifs.

In [ ]:
def display_grid_level(criteria, level_name, level_color):
    """Affiche un niveau de la grille APC sous forme de tableau."""
    data = []
    for criterion in criteria:
        indicators_text = "<br>".join([f"• {ind}" for ind in criterion.indicators])
        data.append({
            "Critère": criterion.description,
            "Indicateurs": indicators_text,
            "Points": criterion.points
        })
    
    df = pd.DataFrame(data)
    
    # Display with styling
    display(Markdown(f"### {level_name}"))
    display(HTML(df.to_html(escape=False, index=False)))
    print()

# Afficher chaque niveau
display(Markdown("## 📊 Grille APC Complète"))
display(Markdown("---"))

display_grid_level(assessment.grid.nd_criteria, "🔴 Niveau ND (Non Développé)", "#ffebee")
display_grid_level(assessment.grid.ni_criteria, "🟡 Niveau NI (Niveau Intermédiaire)", "#fff9c4")
display_grid_level(assessment.grid.na_criteria, "🟢 Niveau NA (Niveau Attendu)", "#e8f5e9")

## 6. Situation d'Évaluation

Affichons la situation d'évaluation authentique générée.

In [ ]:
display(Markdown("## 📝 Situation d'Évaluation Authentique"))
display(Markdown("---"))

display(Markdown(f"### Contexte\n\n{assessment.situation.context}"))
display(Markdown(f"### Tâche\n\n{assessment.situation.task}"))
display(Markdown(f"### Instructions\n\n{assessment.situation.instructions}"))
display(Markdown(f"**Durée** : {assessment.situation.duration}"))

## 7. Barème de Notation

Visualisons le barème de notation avec la répartition des points.

In [ ]:
display(Markdown("## 📈 Barème de Notation"))
display(Markdown("---"))

display(Markdown(f"**Total de points** : {assessment.rubric.total_points}"))
display(Markdown(""))

# Répartition par niveau
display(Markdown("### Répartition par niveau"))
level_data = [
    {"Niveau": "ND (Non Développé)", "Plage de points": f"{assessment.rubric.nd_range[0]} - {assessment.rubric.nd_range[1]}"},
    {"Niveau": "NI (Niveau Intermédiaire)", "Plage de points": f"{assessment.rubric.ni_range[0]} - {assessment.rubric.ni_range[1]}"},
    {"Niveau": "NA (Niveau Attendu)", "Plage de points": f"{assessment.rubric.na_range[0]} - {assessment.rubric.na_range[1]}"}
]
df_levels = pd.DataFrame(level_data)
display(HTML(df_levels.to_html(index=False)))

# Points par critère
display(Markdown("### Points par critère"))
criteria_data = [{"Critère": k, "Points": v} for k, v in assessment.rubric.criteria_points.items()]
df_criteria = pd.DataFrame(criteria_data)
display(HTML(df_criteria.to_html(index=False)))

# Vérification de la somme
total_assigned = sum(assessment.rubric.criteria_points.values())
display(Markdown(f"\n**Vérification** : Somme des points = {total_assigned} / {assessment.rubric.total_points}"))

## 8. Validation par l'Agent Critique

Utilisons l'agent critique pour valider la cohérence pédagogique de la grille générée.

⏱️ Cette étape prend généralement 5-10 secondes.

In [ ]:
# Initialiser l'agent critique
critic = CriticAgent(api_key)

print("🔍 Validation de la qualité pédagogique en cours...")
print("   Cela peut prendre 5-10 secondes...\n")

# Valider l'évaluation
validation = critic.validate_assessment(assessment)

print("\n✅ Validation terminée!")

if validation.is_valid:
    print("   ✅ Validation réussie!")
    if validation.alignment_score:
        print(f"   Score d'alignement : {validation.alignment_score}")
else:
    print("   ⚠️  Validation avec remarques")
    if validation.observability_issues:
        print(f"   - {len(validation.observability_issues)} problème(s) d'observabilité")
    if validation.coherence_issues:
        print(f"   - {len(validation.coherence_issues)} problème(s) de cohérence")

### Résultats de la Validation

In [ ]:
display(Markdown("## ✅ Résultats de la Validation"))
display(Markdown("---"))

# Statut
status_icon = "✅" if validation.is_valid else "⚠️"
status_text = "Validé" if validation.is_valid else "Validation avec remarques"
display(Markdown(f"**Statut** : {status_icon} {status_text}"))

if validation.alignment_score:
    display(Markdown(f"**Score d'alignement** : {validation.alignment_score}"))

# Problèmes d'observabilité
if validation.observability_issues:
    display(Markdown("### Problèmes d'observabilité"))
    for issue in validation.observability_issues:
        display(Markdown(f"- {issue}"))

# Problèmes de cohérence
if validation.coherence_issues:
    display(Markdown("### Problèmes de cohérence"))
    for issue in validation.coherence_issues:
        display(Markdown(f"- {issue}"))

# Feedback général
if validation.feedback:
    display(Markdown("### Feedback"))
    display(Markdown(validation.feedback))

## 9. Export des Résultats

Exportons la grille d'évaluation dans les formats JSON et Markdown.

In [ ]:
# Initialiser le module d'export
exporter = ExportModule()

# Générer un nom de fichier basé sur la compétence
base_filename = competency_input.competency[:50].strip().replace(" ", "_").lower()
base_filename = "".join(c for c in base_filename if c.isalnum() or c in ("_", "-"))

if not base_filename:
    base_filename = "assessment"

print("💾 Export des résultats en cours...\n")

# Export JSON
json_path = exporter.export_json(assessment, validation, f"{base_filename}.json")
print(f"✅ Export JSON : {json_path}")

# Export Markdown
md_path = exporter.export_markdown(assessment, validation, f"{base_filename}.md")
print(f"✅ Export Markdown : {md_path}")

print("\n✨ Export terminé avec succès!")

## 10. Résumé Final

Affichons un résumé complet de la génération.

In [ ]:
display(Markdown("# 🎉 Génération Terminée!"))
display(Markdown("---"))

display(Markdown("## 📊 Résumé"))
display(Markdown(f"**Compétence** : {assessment.input.competency}"))
display(Markdown(f"**Niveau** : {assessment.input.niveau}"))
display(Markdown(f"**Parcours** : {assessment.input.parcours}"))
display(Markdown(f"**Spécialité** : {assessment.input.specialite}"))
display(Markdown(f"**Durée** : {assessment.input.duree}"))
display(Markdown(""))

display(Markdown("## 📁 Fichiers Générés"))
display(Markdown(f"- **JSON** : `{json_path}`"))
display(Markdown(f"- **Markdown** : `{md_path}`"))
display(Markdown(""))

display(Markdown("## ✅ Statistiques"))
display(Markdown(f"- **Critères ND** : {len(assessment.grid.nd_criteria)}"))
display(Markdown(f"- **Critères NI** : {len(assessment.grid.ni_criteria)}"))
display(Markdown(f"- **Critères NA** : {len(assessment.grid.na_criteria)}"))
display(Markdown(f"- **Total de points** : {assessment.rubric.total_points}"))
display(Markdown(f"- **Statut de validation** : {'✅ Validé' if validation.is_valid else '⚠️ Avec remarques'}"))

display(Markdown(""))
display(Markdown("---"))
display(Markdown("*Généré par SkillAssessGPT - Système d'automatisation IA pour grilles APC*"))

## 📚 Exemples Supplémentaires

Vous pouvez réutiliser ce notebook avec d'autres compétences. Voici quelques exemples :

### Exemple 1 : Gestion de Projet

In [ ]:
# Exemple : Gestion de projet
# example_input = CompetencyInput(
#     competency="Gérer un projet informatique de A à Z",
#     element="Planifier et suivre l'avancement d'un projet",
#     niveau="Master 1",
#     parcours="Informatique",
#     specialite="Gestion de Projet",
#     duree="3 heures"
# )

### Exemple 2 : Base de Données

In [ ]:
# Exemple : Base de données
# example_input = CompetencyInput(
#     competency="Concevoir et implémenter une base de données relationnelle",
#     element="Modéliser les données et créer les tables",
#     niveau="Licence 3",
#     parcours="Informatique",
#     specialite="Base de Données",
#     duree="2 heures 30 minutes"
# )

### Exemple 3 : Sécurité Informatique

In [ ]:
# Exemple : Sécurité informatique
# example_input = CompetencyInput(
#     competency="Sécuriser une application web contre les vulnérabilités courantes",
#     element="Implémenter les protections OWASP Top 10",
#     niveau="Master 2",
#     parcours="Informatique",
#     specialite="Cybersécurité",
#     duree="4 heures"
# )

## 🔧 Utilisation Avancée

### Pipeline Complet en Une Seule Cellule

Pour une utilisation rapide, vous pouvez exécuter tout le pipeline en une seule fois :

In [ ]:
# Pipeline complet
def generate_complete_assessment(competency_input, api_key):
    """
    Génère une évaluation complète en une seule fonction.
    
    Args:
        competency_input: CompetencyInput object
        api_key: DeepSeek API key
        
    Returns:
        Tuple of (assessment, validation, json_path, md_path)
    """
    # Génération
    generator = GeneratorAgent(api_key)
    assessment = generator.generate_assessment(competency_input)
    
    # Validation
    critic = CriticAgent(api_key)
    validation = critic.validate_assessment(assessment)
    
    # Export
    exporter = ExportModule()
    base_filename = competency_input.competency[:50].strip().replace(" ", "_").lower()
    base_filename = "".join(c for c in base_filename if c.isalnum() or c in ("_", "-"))
    if not base_filename:
        base_filename = "assessment"
    
    json_path = exporter.export_json(assessment, validation, f"{base_filename}.json")
    md_path = exporter.export_markdown(assessment, validation, f"{base_filename}.md")
    
    return assessment, validation, json_path, md_path

# Exemple d'utilisation :
# assessment, validation, json_path, md_path = generate_complete_assessment(competency_input, api_key)
# print(f"✅ Génération terminée! Fichiers : {json_path}, {md_path}")

## 📖 Documentation

Pour plus d'informations sur SkillAssessGPT :

- **README.md** : Guide d'installation et d'utilisation
- **Design Document** : Architecture et spécifications techniques
- **Requirements Document** : Exigences fonctionnelles détaillées

## 🤝 Support

Pour toute question ou problème :
1. Vérifiez que votre clé API Google Gemini est correctement configurée
2. Assurez-vous que tous les modules sont installés : `pip install -r requirements.txt`
3. Consultez la documentation dans le dossier `.kiro/specs/skillassess-gpt/`